In [49]:
# LSEG/Refinitive Data Acquisition of Required Data
#Target Period: 2006-01-01 to 2025-06-30

#Imports
#Import Refinitiv data library for data acquisition
import pandas as pd
import numpy as np
import yfinance as yf
import refinitiv.data as rd
from refinitiv.data.content import historical_pricing as hp

In [51]:
"""
#Start Refinitiv Session
rd.open_session(app_key="2ad05903f7cb4f84b75e13735fb71abde98f7d85")
"""

'\n#Start Refinitiv Session\nrd.open_session(app_key="2ad05903f7cb4f84b75e13735fb71abde98f7d85")\n'

In [52]:
"""
#Define instruments and Date range
START_DATE = "2006-01-01"
END_DATE = "2025-06-30"

# Replace these placeholders with the actual Refinitiv RICs 
SPX_PROXY_RIC     = "SPY.P"   # S&P 500 ETF (proxy for .SPX)
DOLLAR_INDEX_RIC  = ".DXY"    # Dollar Index
WTI_OIL_RIC       = "CLc1"    # WTI Crude Oil Futures (continuous contract)
"""

'\n#Define instruments and Date range\nSTART_DATE = "2006-01-01"\nEND_DATE = "2025-06-30"\n\n# Replace these placeholders with the actual Refinitiv RICs \nSPX_PROXY_RIC     = "SPY.P"   # S&P 500 ETF (proxy for .SPX)\nDOLLAR_INDEX_RIC  = ".DXY"    # Dollar Index\nWTI_OIL_RIC       = "CLc1"    # WTI Crude Oil Futures (continuous contract)\n'

In [53]:
#import refinitiv.data as rd
#print("Refinitiv Data Library version:", rd.__version__)

In [54]:
#print(dir(hp))

In [55]:
"""
from refinitiv.data.content import pricing

# request latest snapshot for Apple
snap = pricing.Definition(universe="AAPL.O").get_data()
print(snap)
"""

'\nfrom refinitiv.data.content import pricing\n\n# request latest snapshot for Apple\nsnap = pricing.Definition(universe="AAPL.O").get_data()\nprint(snap)\n'

In [56]:
"""
#Obtain required data
# --- 1. S&P 500 OHLCV ---
df_spx_raw = hp.summaries.Definition(
    universe=SPX_PROXY_RIC,
    fields=["OPEN", "HIGH", "LOW", "CLOSE", "VOLUME"],
    interval=hp.Intervals.DAILY,
    start=START_DATE,
    end=END_DATE
).get_data()

# --- 2. Dollar Index & WTI Oil (Closing Prices) ---
df_macro_raw = hp.summaries.Definition(
    universe=[DOLLAR_INDEX_RIC, WTI_OIL_RIC],
    fields=["CLOSE"],
    interval=hp.Intervals.DAILY,
    start=START_DATE,
    end=END_DATE
).get_data()

# Output checks
print("S&P 500 Raw DataFrame (df_spx_raw):")
print(df_spx_raw.head())

print("\nMacro Raw DataFrame (df_macro_raw):")
print(df_macro_raw.head())
"""

'\n#Obtain required data\n# --- 1. S&P 500 OHLCV ---\ndf_spx_raw = hp.summaries.Definition(\n    universe=SPX_PROXY_RIC,\n    fields=["OPEN", "HIGH", "LOW", "CLOSE", "VOLUME"],\n    interval=hp.Intervals.DAILY,\n    start=START_DATE,\n    end=END_DATE\n).get_data()\n\n# --- 2. Dollar Index & WTI Oil (Closing Prices) ---\ndf_macro_raw = hp.summaries.Definition(\n    universe=[DOLLAR_INDEX_RIC, WTI_OIL_RIC],\n    fields=["CLOSE"],\n    interval=hp.Intervals.DAILY,\n    start=START_DATE,\n    end=END_DATE\n).get_data()\n\n# Output checks\nprint("S&P 500 Raw DataFrame (df_spx_raw):")\nprint(df_spx_raw.head())\n\nprint("\nMacro Raw DataFrame (df_macro_raw):")\nprint(df_macro_raw.head())\n'

In [58]:
#Define instruments and Date range
start = "2006-01-01"
end = "2025-06-30"

# S&P500 ETF (proxy)
df_spx_raw = yf.download("SPY", start=start, end=end)

# Dollar Index (Yahoo ticker: DX-Y.NYB)
df_dxy = yf.download("DX-Y.NYB", start=start, end=end)

# WTI Oil Futures (Yahoo ticker: CL=F)
df_wti = yf.download("CL=F", start=start, end=end)

print(df_spx_raw.head())
print(df_dxy.head())
print(df_wti.head())

C:\Users\josep\AppData\Local\Temp\ipykernel_50828\3065575225.py:6:FutureWarning: YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  1 of 1 completed
C:\Users\josep\AppData\Local\Temp\ipykernel_50828\3065575225.py:9:FutureWarning: YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  1 of 1 completed
C:\Users\josep\AppData\Local\Temp\ipykernel_50828\3065575225.py:12:FutureWarning: YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open    Volume
Ticker            SPY        SPY        SPY        SPY       SPY
Date                                                            
2006-01-03  87.721733  87.929443  86.122388  86.676277  73256700
2006-01-04  88.137161  88.268706  87.721742  87.832522  51899600
2006-01-05  88.192497  88.337892  87.846319  88.033258  47307500
2006-01-06  88.926437  89.023367  88.178690  88.635648  62885900
2006-01-09  89.154892  89.355671  88.884873  88.912563  43527400
Price           Close       High        Low       Open   Volume
Ticker       DX-Y.NYB   DX-Y.NYB   DX-Y.NYB   DX-Y.NYB DX-Y.NYB
Date                                                           
2006-01-03  89.839996  90.940002  89.779999  90.750000        0
2006-01-04  89.139999  89.860001  89.010002  89.730003        0
2006-01-05  89.330002  89.629997  89.180000  89.269997        0
2006-01-06  88.849998  89.629997  88.800003  89.349998        0
2006-01-09  89.250000  89.449997